# CAE Notebook

To use this notebook, put the appropriate Chemistry Data in a folder in Google Drive, after mounting the drive it is crucial to select the correct folder, which is the first line of this notebook:

foldername = 'gdrive/MyDrive/'

Edit this to have the correct foldername.

After that it is fairly easy to run. Hyperparameters can be changed, and in the Run Experiment part of the notebook one can run a synthetic or chemistry experiment easily by commenting out which experiment to run later.

In [ ]:
foldername = 'gdrive/MyDrive/'

# CAE - Code from the Concrete Autoencoder Repo

In [ ]:
import math

from keras import backend as K
from keras import Model
from keras.layers import Layer, Softmax, Input
from keras.callbacks import EarlyStopping
from keras.initializers import Constant, glorot_normal
from tensorflow.keras.optimizers import Adam



class ConcreteSelect(Layer):
    
    def __init__(self, output_dim, start_temp = 10.0, min_temp = 0.1, alpha = 0.99999, **kwargs):
        self.output_dim = output_dim
        self.start_temp = start_temp
        self.min_temp = K.constant(min_temp)
        self.alpha = K.constant(alpha)
        super(ConcreteSelect, self).__init__(**kwargs)
        
    def build(self, input_shape):
        self.temp = self.add_weight(name = 'temp', shape = [], initializer = Constant(self.start_temp), trainable = False)
        self.logits = self.add_weight(name = 'logits', shape = [self.output_dim, input_shape[1]], initializer = glorot_normal(), trainable = True)
        super(ConcreteSelect, self).build(input_shape)
        
    def call(self, X, training=None):
        uniform = K.random_uniform(self.logits.shape, K.epsilon(), 1.0)
        gumbel = -K.log(-K.log(uniform))
        temp = K.update(self.temp, K.maximum(self.min_temp, self.temp * self.alpha))
        noisy_logits = (self.logits + gumbel) / temp
        samples = K.softmax(noisy_logits)
        
        discrete_logits = K.one_hot(K.argmax(self.logits), self.logits.shape[1])
        
        self.selections = K.in_train_phase(samples, discrete_logits, training)
        Y = K.dot(X, K.transpose(self.selections))
        
        return Y
    
    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.output_dim)
    
class StopperCallback(EarlyStopping):
    
    def __init__(self, mean_max_target = 0.998):
        self.mean_max_target = mean_max_target
        super(StopperCallback, self).__init__(monitor = '', patience = float('inf'), verbose = 1, mode = 'max', baseline = self.mean_max_target)
    
    def on_epoch_begin(self, epoch, logs = None):
        print('mean max of probabilities:', self.get_monitor_value(logs), '- temperature', K.get_value(self.model.get_layer('concrete_select').temp))
        #print( K.get_value(K.max(K.softmax(self.model.get_layer('concrete_select').logits), axis = -1)))
        #print(K.get_value(K.max(self.model.get_layer('concrete_select').selections, axis = -1)))
    
    def get_monitor_value(self, logs):
        monitor_value = K.get_value(K.mean(K.max(K.softmax(self.model.get_layer('concrete_select').logits), axis = -1)))
        return monitor_value


class ConcreteAutoencoderFeatureSelector():
    
    def __init__(self, K, output_function, num_epochs = 300, batch_size = None, learning_rate = 0.001, start_temp = 10.0, min_temp = 0.1, tryout_limit = 5):
        self.K = K
        self.output_function = output_function
        self.num_epochs = num_epochs
        self.batch_size = batch_size
        self.learning_rate = learning_rate
        self.start_temp = start_temp
        self.min_temp = min_temp
        self.tryout_limit = tryout_limit
        
    def fit(self, X, Y = None, val_X = None, val_Y = None):
        if Y is None:
            Y = X
        assert len(X) == len(Y)
        validation_data = None
        if val_X is not None and val_Y is not None:
            assert len(val_X) == len(val_Y)
            validation_data = (val_X, val_Y)
        
        if self.batch_size is None:
            self.batch_size = max(len(X) // 256, 16)
        
        num_epochs = self.num_epochs
        steps_per_epoch = (len(X) + self.batch_size - 1) // self.batch_size
        
        for i in range(self.tryout_limit):
            
            K.set_learning_phase(1)
            
            inputs = Input(shape = X.shape[1:])

            alpha = math.exp(math.log(self.min_temp / self.start_temp) / (num_epochs * steps_per_epoch))
            
            self.concrete_select = ConcreteSelect(self.K, self.start_temp, self.min_temp, alpha, name = 'concrete_select')

            selected_features = self.concrete_select(inputs)

            outputs = self.output_function(selected_features)

            self.model = Model(inputs, outputs)

            self.model.compile(Adam(self.learning_rate), loss = 'mean_squared_error')
            
            print(self.model.summary())
            
            stopper_callback = StopperCallback()
            
            hist = self.model.fit(X, Y, self.batch_size, num_epochs, verbose = 1, callbacks = [stopper_callback], validation_data = validation_data)#, validation_freq = 10)
            
            if K.get_value(K.mean(K.max(K.softmax(self.concrete_select.logits, axis = -1)))) >= stopper_callback.mean_max_target:
                break
            
            num_epochs *= 2
        
        self.probabilities = K.get_value(K.softmax(self.model.get_layer('concrete_select').logits))
        self.indices = K.get_value(K.argmax(self.model.get_layer('concrete_select').logits))
            
        return self
    
    def get_indices(self):
        return K.get_value(K.argmax(self.model.get_layer('concrete_select').logits))
    
    def get_mask(self):
        return K.get_value(K.sum(K.one_hot(K.argmax(self.model.get_layer('concrete_select').logits), self.model.get_layer('concrete_select').logits.shape[1]), axis = 0))
    
    def transform(self, X):
        return X[self.get_indices()]
    
    def fit_transform(self, X, y):
        self.fit(X, y)
        return self.transform(X)
    
    def get_support(self, indices = False):
        return self.get_indices() if indices else self.get_mask()
    
    def get_params(self):
        return self.model

# Experiment

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense, Softmax
from keras.layers.convolutional.conv1d import regularizers
import numpy as np
import os
import os.path as osp

from functools import reduce


def set_seed(x):
    # set a consistent seed, so we can run across different runs
    x *= 10000
    np.random.seed(x)
    tf.random.set_seed(x)

#Syn Data Specific

In [ ]:
# Logic Rules.

def rule1(x_sample):
    return (x_sample[0] > 0.55) or (x_sample[1] > 0.55)

def rule2(x_sample):
    return (x_sample[0]*x_sample[1] > 0.30) or (x_sample[2]*x_sample[3] > 0.30)

def rule3(x_sample):
    return (x_sample[0]*x_sample[1] > 0.30) or (x_sample[0]*x_sample[2] > 0.30)

def rule4(x_sample):
    return (x_sample[0]*x_sample[3] > 0.30) or (x_sample[6]*x_sample[9] > 0.30)

# Sampling rules

def normal_sample(nsamples, nfeatures):
    return np.random.normal(size=(nsamples, nfeatures))

def correlated_sample(nsamples, nfeatures):
    mean = np.array([0.0, 0.0, 0.0])
    cov = np.array([[1, 0.99, 0.99],
                    [0.99, 1, 0.99],
                    [0.99, 0.99, 1]])

    x123 = np.random.multivariate_normal(mean, cov, size=nsamples)
    x456 = np.random.multivariate_normal(mean, cov, size=nsamples)
    x789 = np.random.multivariate_normal(mean, cov, size=nsamples)
    x101112 = np.random.multivariate_normal(mean, cov, size=nsamples)
    xrest = np.random.normal(size=(nsamples, nfeatures-4*3))
    return np.concatenate([x123, x456, x789, x101112, xrest], axis=1)

sampling_rules = {
        1: normal_sample,
        2: normal_sample,
        3: normal_sample,
        4: correlated_sample
    }

logic_rules = {
    1: rule1,
    2: rule2,
    3: rule3,
    4: rule4
}

gauss_groups = {1: [np.array([0]), np.array([1])], 2: [np.array([0, 1]), np.array([2, 3])],
                3: [np.array([0, 1]), np.array([0, 2])], 4: [np.array([0, 3]), np.array([6, 9])]}
gauss_oracle_features = {1: np.array([0, 1]), 2: np.array([0, 1, 2, 3]),
                         3: np.array([0, 1, 2]), 4: np.array([0, 3, 6, 9])}



def make_syn_data(rule, nsamples, nfeatures, train):
    x_data = sampling_rules[rule](nsamples, nfeatures)
    y_data = np.array([logic_rules[rule](x) for x in x_data])
    return x_data, y_data

# Chem Data Specific

In [ ]:
from google.colab import drive
drive.mount('gdrive')


chem_data_groups = {4: [np.array([40]), np.array([1])], # logic_4 = ether OR NOT alkyne
		            10: [np.array([56, 18]), np.array([40])], # logic_10 = (primary amine AND NOT ether) OR (NOT benzene AND NOT ether)
		            13: [np.array([18, 29]), np.array([1, 40])], # logic_13 = (benzene AND NOT carbonyl) OR (alkyne AND NOT ether)
                    }
chem_oracle_features = {4: np.array([1, 40]),
                        10: np.array([18, 40, 56]),
                        13: np.array([1, 18, 29, 40])}


def make_chem_data(rule, train=True):
    is_train = 'train' if train else 'test'
    x_data = np.load(osp.join(foldername, 'logic_'+str(rule)+'_X_'+is_train+'.npy'))
    y_data = np.load(osp.join(foldername, 'logic_'+str(rule)+'_Y_'+is_train+'.npy'))
    return x_data, y_data




Mounted at gdrive


# Metrics

In [ ]:
def gsim(true_groups, predicted_groups):
    # Returns gsim, number of true groups, and number of discovered groups, given
    # true groups and predicted groups as input.
    gsim = 0
    if len(true_groups) == 0: # i.e. we don't know the ground truth.
       return -1, len(true_groups), len(predicted_groups)
    if len(predicted_groups)>0:
      for g in true_groups:
         current_max = 0
         for g_hat in predicted_groups:
            jac = np.intersect1d(g, g_hat).size / np.union1d(g, g_hat).size
            if jac == 1:
               current_max = 1
               break
            if jac > current_max:
               current_max = jac
         gsim += current_max
      gsim /= max(len(true_groups), len(predicted_groups))
      return gsim, len(true_groups), len(predicted_groups)
    else:   # We didn't find anything.
      return 0, len(true_groups), len(predicted_groups)


def tpr_fdr(true_groups, predicted_groups):
   # True positive rate and false discovery rate.
   
   if len(true_groups) == 0:  # Ground truth not known.
      return -1, -1

   if len(predicted_groups) == 0:
      return 0.0, 0.0

   predicted_features = np.unique(reduce(np.union1d, predicted_groups))
   true_features = np.unique(reduce(np.union1d, true_groups))

   overlap = np.intersect1d(predicted_features, true_features).size
   tpr = 100*overlap/len(true_features)
   fdr = 100*(len(predicted_features)-overlap)/len(predicted_features) # If len(predicted_features) != 0 else 0.0.
   return tpr, fdr

# Setup Data

Edit the first cell below to choose which data to use.

In [ ]:
# Edit these lines, choice of experiment is syn or chem

choice = 'syn'     # Uncomment one of these 
#choice = 'chem'     # Uncomment one of these
rule = 2
experiment_no = 1

In [ ]:
set_seed(experiment_no)

if choice == 'syn':
    train_size = 20000
    test_size = 200
    nfeatures = 500
    nhidden = 200
    batchsize = 250
    lr = 0.001
    nepochs = 250
    tryoutlimit = 1
    nk = len(gauss_oracle_features[rule])


    x_train, y_train_ = make_syn_data(rule, train_size, nfeatures, train=True)
    x_test, y_test_ = make_syn_data(rule, test_size, nfeatures, train=False)
    y_train = to_categorical(y_train_)
    y_test = to_categorical(y_test_)
    true_groups = gauss_groups[rule]

if choice == 'chem':
    nhidden = 200
    batchsize = 250
    lr = 0.001
    nepochs = 250
    tryoutlimit = 1

    rules = {
        1: 4,
        2: 10,
        3: 13
    }
    rule = rules[rule]

    nk = len(chem_oracle_features[rule])
    

    x_train, y_train_ = make_chem_data(rule, train=True)
    x_test, y_test_ = make_chem_data(rule, train=False)
    y_train = to_categorical(y_train_)
    y_test = to_categorical(y_test_)
    nfeatures = x_train.shape[-1]
    true_groups = chem_data_groups[rule]
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(20000, 500)
(20000, 2)
(200, 500)
(200, 2)


# Train Models

In [ ]:
final_activation = 'sigmoid' if choice == 'chem' else 'linear' 
def f(x):
    x = Dense(nhidden, activation='relu')(x)
    x = Dense(nhidden, activation='relu')(x)
    x = Dense(nfeatures, activation=final_activation)(x)
    return x

def g(x):
    x = Dense(nhidden, activation='relu')(x)
    x = Dense(nhidden, activation='relu')(x)
    x = Dense(2)(x)
    x = Softmax()(x)
    return x

In [ ]:
supervised_selector = ConcreteAutoencoderFeatureSelector(K=nk, output_function=g, num_epochs=nepochs, tryout_limit=tryoutlimit, batch_size=batchsize)
supervised_selector.fit(x_train, y_train, x_test, y_test)

Model: "model_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_9 (InputLayer)        [(None, 500)]             0         
                                                                 
 concrete_select (ConcreteSe  (None, 4)                2001      
 lect)                                                           
                                                                 
 dense_24 (Dense)            (None, 200)               1000      
                                                                 
 dense_25 (Dense)            (None, 200)               40200     
                                                                 
 dense_26 (Dense)            (None, 2)                 402       
                                                                 
 softmax_4 (Softmax)         (None, 2)                 0         
                                                           

In [ ]:
unsupervised_selector = ConcreteAutoencoderFeatureSelector(K=nk, output_function=f, num_epochs=nepochs, tryout_limit=tryoutlimit, batch_size=batchsize)
unsupervised_selector.fit(x_train, x_train, x_test, x_test)

Model: "model_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_10 (InputLayer)       [(None, 500)]             0         
                                                                 
 concrete_select (ConcreteSe  (None, 4)                2001      
 lect)                                                           
                                                                 
 dense_27 (Dense)            (None, 200)               1000      
                                                                 
 dense_28 (Dense)            (None, 200)               40200     
                                                                 
 dense_29 (Dense)            (None, 500)               100500    
                                                                 
Total params: 143,701
Trainable params: 143,700
Non-trainable params: 1
_____________________________________________________

# Evaluate Models

In [ ]:
y_pred = supervised_selector.model(x_test)
y_pred = tf.math.argmax(y_pred, axis=-1)
acc = 100*np.mean((y_pred==y_test_).numpy())

x_pred = unsupervised_selector.model(x_test)
reconstruction_error = np.mean((x_pred.numpy() - x_test)**2)

In [ ]:
supervised_selected = supervised_selector.get_indices()
unsupervised_selected = unsupervised_selector.get_indices()

selected = np.unique(supervised_selected)
tpr, fdr = tpr_fdr(true_groups, [selected])
group_similarity, num_true_groups, num_discovered_groups = gsim(true_groups, [selected])

print('Supervised:')
print('Accuracy: {:.3f}%'.format(acc))
print('Selected features: {}'.format(selected))
print('TPR: {:.3f}%'.format(tpr))
print('FDR: {:.3f}%'.format(fdr))
print('Gsim: {:.3f}'.format(group_similarity))
print('Num True Groups: {}'.format(num_true_groups))
print('Num Discovered Groups: {}'.format(num_discovered_groups))


selected = np.unique(unsupervised_selected)
tpr, fdr = tpr_fdr(true_groups, [selected])
group_similarity, num_true_groups, num_discovered_groups = gsim(true_groups, [selected])

print('\nUnsupervised:')
print('Reconstruction Error: {:.3f}'.format(reconstruction_error))
print('Selected features: {}'.format(selected))
print('TPR: {:.3f}%'.format(tpr))
print('FDR: {:.3f}%'.format(fdr))
print('Gsim: {:.3f}'.format(group_similarity))
print('Num True Groups: {}'.format(num_true_groups))
print('Num Discovered Groups: {}'.format(num_discovered_groups))

Supervised:
Accuracy: 47.500%
Selected features: [162 272 310]
TPR: 0.000%
FDR: 100.000%
Gsim: 0.000
Num True Groups: 2
Num Discovered Groups: 1

Unsupervised:
Reconstruction Error: 0.996
Selected features: [104 184 225 285]
TPR: 0.000%
FDR: 100.000%
Gsim: 0.000
Num True Groups: 2
Num Discovered Groups: 1
